In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

from pathlib import Path
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import json


from IPython.display import display
import pandas_profiling
from pandas_profiling import ProfileReport
%matplotlib inline

from lightfm import LightFM



print(os.path.abspath(''))
root_path = os.path.abspath('')+"/../"
print(root_path)
data_folder = root_path+"/data/"

#train = data['train']
#test = data['test']

C:\Users\samir\Documents\Python Scripts\kaggle-retail-rocket\notebooks
C:\Users\samir\Documents\Python Scripts\kaggle-retail-rocket\notebooks/../


C:\Users\samir\anaconda3\envs\retail-rocket\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
### read users interaction
events = pd.read_csv(data_folder+"events.csv", header= 0)
events_reduced = events.drop_duplicates()
events_final = events_reduced.drop_duplicates(['visitorid', 'event', 'itemid']).copy()

#events_tupel = []
#events_final = events_reduced.sort_values(by='timestamp', ascending=True).copy()
#events_final.loc[events_final['event'] == 'view', 'rating'] = 2
#events_final.loc[events_final['event'] == 'addtocart', 'rating'] = 3
#events_final.loc[events_final['event'] == 'transaction', 'rating'] = 5
#for index, row in events_final.iterrows():
#    events_tupel.append((row['visitorid'], row['itemid'], row['rating']))



In [14]:

#with open(data_folder+'/events_rating_tupel.json', 'w') as fp:
#    json.dump(events_tupel, fp, default=myconverter)
events_tupel = []
with open(data_folder+'/events_rating_tupel_1.json', 'r') as fp:
    events_tupel = json.load(fp)

In [15]:
#print(train)

start_index = int(0.2*len(events_tupel))
end_index = int(0.3*len(events_tupel))
events_tupel = events_tupel[start_index:end_index]

total_events = len(events_tupel)

train_amt = int(0.9*total_events)
events_train = events_tupel[:train_amt]
events_test = events_tupel[(train_amt+1):]

print(len(events_train))
print(len(events_test))

193066
21451


In [16]:
#print(test)
from lightfm.data import Dataset

dataset = Dataset(user_identity_features=False, item_identity_features=False)
dataset.fit(events_final['visitorid'].to_numpy(), events_final['itemid'].to_numpy())

(interactions_train, weights_train) = dataset.build_interactions(events_train)
(interactions_test, weights_test) = dataset.build_interactions(events_test)


In [17]:
# Set the number of threads; you can increase this
# if you have more physical cores available.
NUM_THREADS = 2
NUM_COMPONENTS = 30
NUM_EPOCHS = 3
ITEM_ALPHA = 1e-6

# Let's fit a WARP model: these generally have the best performance.
model_wrp = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS)
# Run epochs and time it.
%time model_wrp = model_wrp.fit(interactions_train, epochs=NUM_EPOCHS, num_threads=NUM_THREADS)

 

Wall time: 11.6 s


In [18]:

# Let's fit a WARP model: these generally have the best performance.
model_bpr = LightFM(loss='bpr',
                item_alpha=ITEM_ALPHA,
               no_components=NUM_COMPONENTS)
# Run epochs and time it.
%time model_bpr = model_bpr.fit(interactions_train, epochs=NUM_EPOCHS, num_threads=NUM_THREADS)


Wall time: 15.2 s


In [8]:
from lightfm.evaluation import auc_score

# Compute and print the AUC score
%time train_auc = auc_score(model_wrp, interactions_train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

Wall time: 47min 1s
Collaborative filtering train AUC: 0.9357083


Wall time: 47min 1s
Collaborative filtering train AUC: 0.9357083

In [9]:

# Compute and print the AUC score
%time train_auc = auc_score(model_bpr, interactions_train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

Wall time: 13h 14min 38s
Collaborative filtering train AUC: 0.51629496


Wall time: 13h 14min 38s
Collaborative filtering train AUC: 0.51629496

In [19]:
print(interactions_test)
# Compute and print the AUC score
%time train_auc = auc_score(model_wrp, interactions_test, train_interactions=interactions_train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

  (114640, 64870)	1
  (103994, 2881)	1
  (114603, 63752)	1
  (100640, 2595)	1
  (103903, 60961)	1
  (67093, 31227)	1
  (111564, 1887)	1
  (103882, 64835)	1
  (111568, 63669)	1
  (54166, 257)	1
  (103861, 17245)	1
  (112794, 21797)	1
  (103941, 7804)	1
  (114626, 64865)	1
  (103973, 12770)	1
  (103876, 2687)	1
  (112905, 3457)	1
  (106586, 33457)	1
  (114569, 64844)	1
  (109581, 35693)	1
  (104042, 60997)	1
  (93712, 57215)	1
  (103928, 60967)	1
  (103902, 28620)	1
  (103998, 17297)	1
  :	:
  (138941, 73358)	1
  (136641, 31179)	1
  (137832, 4261)	1
  (132249, 179)	1
  (135411, 72122)	1
  (132194, 73336)	1
  (135386, 72117)	1
  (134054, 16465)	1
  (132269, 47526)	1
  (132214, 16786)	1
  (132296, 63959)	1
  (132244, 36521)	1
  (136656, 3976)	1
  (126206, 36551)	1
  (77125, 49929)	1
  (134054, 63300)	1
  (132295, 71171)	1
  (136641, 35138)	1
  (79194, 72107)	1
  (135334, 4785)	1
  (133958, 71655)	1
  (132181, 50966)	1
  (136723, 6899)	1
  (136702, 35812)	1
  (79194, 51141)	1
Wall time: 12m

Wall time: 12min 25s
Collaborative filtering train AUC: 0.7696758

In [20]:

# Compute and print the AUC score
%time train_auc = auc_score(model_bpr, interactions_test, train_interactions=interactions_train, num_threads=NUM_THREADS).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

Wall time: 11min 10s
Collaborative filtering train AUC: 0.50512016


Wall time: 11min 10s
Collaborative filtering train AUC: 0.50512016